In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Parameters
SPEED_THRESHOLD = 4  # m/s
MIN_STOP_DURATION = 1.0  # seconds
DIST_THRESH = 2.8
EXPORT_INTERVAL = 0.16  # seconds
PED_BOX = (0.3, 0.3)
VAN_BOX = (4.48,1.80)

# ----------------------------------
# Helpers
# ----------------------------------

def filter_stopped_vehicles(df, speed_col='speed', time_col='TimeStamp'):
    """Filter out stopped vehicles longer than threshold"""
    df = df.copy()
    df['time_diff'] = df.groupby('Track ID')[time_col].diff()
    df['is_stopped'] = df[speed_col] < SPEED_THRESHOLD
    stop_groups = (df['is_stopped'] != df['is_stopped'].shift()).cumsum()
    df['stop_duration'] = df.groupby(['Track ID', stop_groups])['time_diff'].cumsum()
    return df[~((df['is_stopped']) & (df['stop_duration'] >= MIN_STOP_DURATION))].drop(columns=['time_diff', 'is_stopped', 'stop_duration'])

def get_rotated_corners(x, y, heading, length, width):
    """Return coordinates of rotated bounding box corners"""
    half_l, half_w = length / 2, width / 2
    corners = np.array([
        [ half_l,  half_w],
        [ half_l, -half_w],
        [-half_l, -half_w],
        [-half_l,  half_w]
    ])
    rad = np.radians(heading)
    rot = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad),  np.cos(rad)]
    ])
    return (corners @ rot.T) + np.array([x, y])

def get_closest_corners(row):
    """Get direction vector and min distance between closest corners"""
    ped_corners = get_rotated_corners(row['x_smooth_ped'], row['y_smooth_ped'], row['HA_ped'], *PED_BOX)
    van_corners = get_rotated_corners(row['x_smooth_van'], row['y_smooth_van'], row['HA_van'], *VAN_BOX)
    dists = cdist(ped_corners, van_corners)
    idx = np.unravel_index(np.argmin(dists), dists.shape)
    min_dist = dists[idx]
    direction_vec = van_corners[idx[1]] - ped_corners[idx[0]]
    return min_dist, direction_vec

def calculate_attc(row):
    """Compute ATTC using projection of relative motion on direction vector"""
    min_dist, direction_vec = get_closest_corners(row)
    norm = np.linalg.norm(direction_vec)
    if norm == 0:
        return np.inf
    unit_vec = direction_vec / norm

    rel_v = np.array([
        row['vx_smooth_ped'] - row['vx_smooth_van'],
        row['vy_smooth_ped'] - row['vy_smooth_van']
    ])
    rel_a = 0.5 * np.array([
        row['ax_ped'] - row['ax_van'],
        row['ay_ped'] - row['ay_van']
    ])

    closing_rate = -np.dot(rel_v, unit_vec) + np.dot(rel_a, unit_vec)
    return min_dist / closing_rate if closing_rate > 0 else np.inf

# ----------------------------------
# Load Data
# ----------------------------------

df_ped = pd.read_csv(r"D:\T\test_codeEVT\nd\ped_smooth.csv")
df_van = pd.read_csv(r"D:\T\test_codeEVT\nd\van_smooth.csv")
df_van['speed'] = np.sqrt(df_van['vx_smooth']**2 + df_van['vy_smooth']**2)

# Filter stopped vehicles
print(f"Original vanrcycle count: {len(df_van)}")
df_van = filter_stopped_vehicles(df_van, speed_col='speed')
print(f"Filtered vanrcycle count: {len(df_van)}")

# Calculate yaw rates
for df in [df_ped, df_van]:
    df['yaw_rate'] = df.groupby('Track ID').apply(
        lambda x: x['HA'].diff() / x['TimeStamp'].diff()
    ).reset_index(level=0, drop=True)

# Round timestamps
df_ped['Time_rounded'] = (df_ped['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL
df_van['Time_rounded'] = (df_van['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL


Original vanrcycle count: 3333
Filtered vanrcycle count: 269


C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_22116\2808504400.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(
C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_22116\2808504400.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(


In [3]:

# ----------------------------------
# Merge and Process
# ----------------------------------

merged = pd.merge(
    df_ped, 
    df_van, 
    on='Time_rounded', 
    suffixes=('_ped', '_van')
)

merged['Center_dist'] = np.hypot(
    merged['x_smooth_ped'] - merged['x_smooth_van'],
    merged['y_smooth_ped'] - merged['y_smooth_van']
)

# Filter by center threshold
results = merged[merged['Center_dist'] <= DIST_THRESH].copy()

# Calculate ATTC using corrected method
results['ATTC'] = results.apply(calculate_attc, axis=1)
results[['mindis', 'direction_vec']] = results.apply(
    lambda row: pd.Series(get_closest_corners(row)),
    axis=1
)# Output
output = results[['Track ID_ped', 'Track ID_van', 'TimeStamp_ped', 'ATTC']].rename(columns={
    'Track ID_ped': 'Ped_ID',
    'Track ID_van': 'van_ID',
    'TimeStamp_ped': 'TimeStamp'
})
print(output)
len(results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_van']].drop_duplicates())

     Ped_ID  van_ID  TimeStamp       ATTC
792   13065   13134    4538.48        inf
793   13065   13134    4538.64        inf
794   13065   13134    4538.80        inf
795   13065   13134    4538.80        inf
796   13065   13134    4538.96        inf
797   13065   13134    4538.96        inf
798   13065   13134    4539.12   7.843624
799   13065   13134    4539.12   7.843624
800   13065   13134    4539.28   2.099076
801   13065   13134    4539.28   2.099076
802   13065   13134    4539.44        inf
869   14681   14685    5006.36        inf
870   14681   14685    5006.52  99.629585
871   14681   14685    5006.68  10.265703
872   14681   14685    5006.84   7.184327
873   14681   14685    5007.00   2.588614
874   14681   14685    5007.16  44.547558
875   14681   14685    5007.32        inf


0

In [4]:
len(results[results['ATTC']<1][['Track ID_ped', 'Track ID_van']].drop_duplicates())

0

In [5]:
# Create new DataFrame with selected and renamed columns
conflict_summary = results[[
    'Track ID_ped',        # Pedestrian ID
    'Track ID_van',       # Vehicle ID
    'Type_van',           # Vehicle type
    'TimeStamp_ped',       # Timestamp
    'mindis',              # Minimum corner distance
    'ATTC'                 # ATTC value
]].copy()

# Rename columns
conflict_summary.rename(columns={
    'Track ID_ped': 'PED_ID',
    'Track ID_van': 'VEH_ID',
    'Type_van': 'TYPE',
    'TimeStamp_ped': 'TIMESTAMP',
    'mindis': 'MIN_COR_DIS',
    'ATTC': 'ATTC'
}, inplace=True)

# Preview
print(conflict_summary.head())


     PED_ID  VEH_ID  TYPE  TIMESTAMP  MIN_COR_DIS  ATTC
792   13065   13134   Van    4538.48          NaN   inf
793   13065   13134   Van    4538.64     0.228551   inf
794   13065   13134   Van    4538.80     0.274500   inf
795   13065   13134   Van    4538.80     0.274500   inf
796   13065   13134   Van    4538.96     0.285275   inf


In [6]:
conflict_summary.to_csv(r"D:\T\test_codeEVT\ATTC_Data/Van_Ped.csv", index=False)